# **Baseline CSV Dataset**

## Importação de pacotes e leitura dos dados

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from utils import get_train_val_set_data


### Leitura dos dados e separação em treino, validação e test

In [2]:
dataset = pd.read_csv('../dataset/Data/features_30_sec.csv')

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = get_train_val_set_data(dataset)

-- Treino --
X (shape):  (700, 59)
y:  700
-- Validação --
X (shape):  (150, 59)
y:  150
-- Teste --
X (shape):  (150, 59)
y:  150


In [4]:
X_train.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,1.297741e+05,2002.449060,85882.761315,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,1.843559e+05,1596.412872,166441.494769,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
2,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,3.433999e+05,1748.172116,88445.209036,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160
3,blues.00005.wav,661794,0.302456,0.087532,0.103494,0.003981,1831.993940,1.030482e+06,1729.653287,201910.508633,...,-5.576589,72.549225,-1.838263,68.702026,-2.783800,42.447453,-3.047909,39.808784,-8.109991,46.311005
4,blues.00006.wav,661794,0.291328,0.093981,0.141874,0.008803,1459.366472,4.378594e+05,1389.009131,185023.239545,...,-10.068051,83.248245,-10.913176,56.902153,-6.971336,38.231800,-3.436505,48.235741,-6.483466,70.170364


## Implementação do baseline

In [6]:
features = dataset.columns.drop(['filename','label'])
X_train = X_train.drop(columns= 'filename')
X_val = X_val.drop(columns= 'filename')
X_test = X_test.drop(columns= 'filename')

---
**PERGUNTA - É um problema desbalanceado?**: 

Para o problema proposto, nota-se que as classes no treino estão bem distribuídas entre si. Assim, não identifica-se um problema desbalanceado

In [7]:
y_train.value_counts(normalize = True)

classical    0.1
hiphop       0.1
pop          0.1
country      0.1
reggae       0.1
blues        0.1
disco        0.1
metal        0.1
rock         0.1
jazz         0.1
Name: label, dtype: float64

---
**PERGUNTA - Os dados precisarão de algum tipo de pré-processamento?**: 

No caso do arquivo csv será necessário realizar apenas a normalização pois nosso baseline trata-se de uma SVM. Para os arquivos de imagem, é necessário fazer a extração de cores por pixel. Por fim, no caso dos arquivos de áudio, é necessário convertê-los em alguma forma interpretável como uma sequência de inteiros por frame da música.

### 5. **Implementação do Baseline**

Como Baseline, optou-se pelo uso de uma SVM para classificação. No caso, não foram implementadas otimizações de hiperparâmetros. A escolha da SVM foi feita pois o número de amostras disponíveis não é alto e dito algoritmo é robusto a um alto número de features

In [8]:
from sklearn.svm import SVC
from sklearn import metrics

In [9]:
clfBaseline = SVC()

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

scaledXtrain = scaler.transform(X_train)
scaledXval = scaler.transform(X_val)

In [11]:
clfBaseline.fit(scaledXtrain, y_train)

SVC()

---
**PERGUNTA - Quais as métricas utilizadas neste problema? Acurácia, erro, MSRE, etc.? O que elas significam?**: 

Para o problema proposto é possível a aplicação de métricas como acurácia balanceada, recall, precisão e f1-score. O significado de cada uma é apresentado abaixo:

1.   Acurácia: Média de TP/(TP + TN + FP + FN) entre as classes
2.   Recall: TP / FN + TP
3.   Precisão: TP / TP + FP
4.   F1-Score: Média harmônica de recall e precisão



In [12]:
resultados = clfBaseline.predict(scaledXval)
print(metrics.classification_report(y_val,resultados))

              precision    recall  f1-score   support

       blues       0.79      0.73      0.76        15
   classical       1.00      1.00      1.00        15
     country       0.92      0.80      0.86        15
       disco       0.50      0.53      0.52        15
      hiphop       0.79      0.73      0.76        15
        jazz       0.82      0.93      0.87        15
       metal       0.69      0.73      0.71        15
         pop       0.92      0.73      0.81        15
      reggae       0.65      0.73      0.69        15
        rock       0.56      0.60      0.58        15

    accuracy                           0.75       150
   macro avg       0.76      0.75      0.76       150
weighted avg       0.76      0.75      0.76       150



In [13]:
print(metrics.balanced_accuracy_score(y_val,resultados))

0.7533333333333332


Avaliando os resultados acima, nota-se um resultado muito satisfatório para o Baseline. As métricas de acurácia, recall e precisão ficaram em torno de ~0.7 o que é muito positivo dado o grau de complexidade limitado para o modelo.